# Torus one-point functions of the $O(n)$ model

In [14]:
Pkg.activate("."); # activate the parent environment
using BootstrapVirasoro, BootstrapVirasoro.LoopModels, BenchmarkTools
println("Number of threads: $(Threads.nthreads())")

Number of threads: 4


  Activating project at `~/Documents/Recherche/projet_these/code/BootstrapVirasoro/examples`


In [15]:
const Sig = Channels{Rational}

# Spectrum of O(n) model
c = CC(β=sqrt(big"2")/(big"0.8"+big"0.2"*im))
diag_field = Field(c, r=0, s=big"0.4"+big"0.1"*im)
ndiag_indices = [(r, s) for r in 1//2:1//2:20 for s in -1+1//(2r):1//(2r):1 if r * s % 1 == 0]
fields = vcat([Field(c, r = r, s = s) for (r, s) in ndiag_indices], diag_field)

function precompute_blocks(indices, fields; parity, precision=10)
    setprecision(BigFloat, floor(Int, 1.2*precision), base=10)
    Δmax = floor(Int, 0.75*precision)
    co = Correlation(Field(c, r=indices[1], s=indices[2]), Δmax)
    parity != 0 && (fields = filter(V -> V.diagonal || V.s >= 0, fields))
    f = V -> IBlock(co, V, parity=parity)
    spec = ChanSpec(co, fields, f)
    return @channels spec
end

function compute_diagblocks(specs, Ps, chan)
    c = specs.s.corr.fields[1].c
    return [IBlock(specs.s.corr, chan, Field(c, P=P)) for P in Ps]
end

# Solve crossing symmetry for given signature
# Optionnally for many different diagonal blocks in the channel
function solve(
        specs, signature;
        rmax=3, show=true, 
        Ps=nothing, diagchan=nothing,
        fix=nothing
    )
    specs = @channels filter(
        V -> V.diagonal && signature[chan] == 0 || !V.diagonal && V.r >= signature[chan],
        specs[chan]
    )
    if Ps === nothing
        res = solve_bootstrap(specs, fix=fix)
        if show
            Base.show(stdout, res.str_cst, rmax=rmax)
        end
    else
        diagblocks = compute_diagblocks(specs, Ps, diagchan)
        res = solve_bootstrap_manyP(specs, diagblocks)
    end
    return res
end

solve (generic function with 1 method)

## r = 0

In [16]:
ind = (0, sqrt(big"2")*(big"0.4"+big"0.2"*im))
blocks = precompute_blocks(ind, fields, parity=1, precision=10);

In [17]:
solve(blocks, Sig(0, 0, 0), rmax=2);

 Channel s
┌─────────────────┬─────────────────────────┬────────────────┐
│ Field           │ Structure constant      │ Relative error │
├─────────────────┼─────────────────────────┼────────────────┤
│ (P=0.11+0.06im) │               1.0+0.0im │            0.0 │
│       (1//2, 0) │      0.416666+0.12037im │     3.43924e-8 │
│          (1, 0) │      0.2517-0.0896112im │     1.34389e-5 │
│          (1, 1) │  -0.00288697-0.315724im │     6.40597e-6 │
│       (3//2, 0) │ -0.00610243-0.0662204im │       0.536567 │
│    (3//2, 2//3) │      -0.15139-0.24163im │      0.0994394 │
│          (2, 0) │      -13321.6-24327.4im │       0.797584 │
│       (2, 1//2) │      -35659.4-14462.9im │       0.862812 │
│          (2, 1) │      -41252.6-8397.12im │       0.873961 │
└─────────────────┴─────────────────────────┴────────────────┘
 Channel t
┌─────────────────┬────────────────────────┬────────────────┐
│ Field           │ Structure constant     │ Relative error │
├─────────────────┼────────────────

## r = 1

In [18]:
blocks = precompute_blocks((1, 0), fields, parity=1, precision=20);

### Signature $(0, 0, 0)$

In [19]:
solve(blocks, Sig(0, 0, 0), rmax=3//2);

 Channel s
┌─────────────────┬────────────────────┬────────────────┐
│ Field           │ Structure constant │ Relative error │
├─────────────────┼────────────────────┼────────────────┤
│ (P=0.11+0.06im) │          1.0+0.0im │            0.0 │
│       (1//2, 0) │   6.2154-4.35819im │    8.87451e-13 │
│          (1, 0) │  5.84666-10.1272im │    7.68521e-11 │
│          (1, 1) │ -4.14065-1.37768im │    5.32785e-11 │
│       (3//2, 0) │  1.45015-8.15207im │     1.45459e-6 │
│    (3//2, 2//3) │  -12.1188+4.3916im │     5.01436e-7 │
└─────────────────┴────────────────────┴────────────────┘
 Channel t
┌─────────────────┬────────────────────┬────────────────┐
│ Field           │ Structure constant │ Relative error │
├─────────────────┼────────────────────┼────────────────┤
│ (P=0.11+0.06im) │  1.0+1.07993e-13im │    1.02386e-13 │
│       (1//2, 0) │   6.2154-4.35819im │    8.15669e-13 │
│          (1, 0) │  5.84666-10.1272im │    3.70241e-11 │
│          (1, 1) │ -4.14065-1.37768im │    3.5105

## r = 2

In [20]:
blocks = precompute_blocks((2, 0), fields, parity=1, precision=25);

### Signature $(\frac12, \frac12, \frac12)$

In [21]:
solve(blocks, Sig(1//2, 1//2, 1//2), rmax=3//2);

 Channel s
┌──────────────┬────────────────────┬────────────────┐
│ Field        │ Structure constant │ Relative error │
├──────────────┼────────────────────┼────────────────┤
│    (1//2, 0) │          1.0+0.0im │            0.0 │
│       (1, 0) │ -34.9208-45.5559im │     3.6251e-13 │
│       (1, 1) │  31.0888+25.0735im │    3.59439e-13 │
│    (3//2, 0) │ -589.454+35.4061im │    6.61775e-10 │
│ (3//2, 2//3) │  277.297-26.5527im │     1.33174e-9 │
└──────────────┴────────────────────┴────────────────┘
 Channel t
┌──────────────┬────────────────────┬────────────────┐
│ Field        │ Structure constant │ Relative error │
├──────────────┼────────────────────┼────────────────┤
│    (1//2, 0) │  1.0+4.58897e-14im │    1.54374e-14 │
│       (1, 0) │ -34.9208-45.5559im │    6.25701e-14 │
│       (1, 1) │  31.0888+25.0735im │    1.25781e-13 │
│    (3//2, 0) │ -589.454+35.4061im │    1.15658e-11 │
│ (3//2, 2//3) │  277.297-26.5527im │    5.00024e-11 │
└──────────────┴────────────────────┴──────

### Signature $(0, 0, 0)$, $D^{(s)}_{(\frac12, 0)} = 0$

In [22]:
fix = [(:s, diag_field, 1.0), (:s, Field(c, r=1//2, s=0), 0.0)]
solve(blocks, Sig(0, 0, 0), fix=fix, rmax=3);

 Channel s
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relative error │
├─────────────────┼────────────────────────────┼────────────────┤
│ (P=0.11+0.06im) │                  1.0+0.0im │            0.0 │
│       (1//2, 0) │                  0.0+0.0im │            0.0 │
│          (1, 0) │         -934.438+210.945im │    1.98011e-13 │
│          (1, 1) │         -597.189+296.177im │    1.83608e-13 │
│       (3//2, 0) │          1538.45+4136.88im │     1.71271e-9 │
│    (3//2, 2//3) │          1587.23+3847.92im │     1.26839e-9 │
│          (2, 0) │          12114.5-2452.02im │     0.00136152 │
│       (2, 1//2) │          3438.54-30959.3im │    0.000479461 │
│          (2, 1) │          11198.4-2765.52im │     0.00123378 │
│       (5//2, 0) │  4.07551e+08+8.40487e+08im │        1.10377 │
│    (5//2, 2//5) │  6.43446e+08+7.50261e+08im │        1.10006 │
│    (5//2, 4//5) │  7.85953e+08+6.64319e+08im │        1.12477 │

## r = 3

In [23]:
blocks = precompute_blocks((3, 0), fields, parity=1, precision=20);

### Signature $(1, 1, 1)$

In [24]:
solve(blocks, Sig(1, 1, 1), rmax=2);

 Channel s
┌──────────────┬──────────────────────┬────────────────┐
│ Field        │ Structure constant   │ Relative error │
├──────────────┼──────────────────────┼────────────────┤
│       (1, 0) │            1.0+0.0im │            0.0 │
│       (1, 1) │ -1.12641-0.0491036im │    8.92952e-14 │
│    (3//2, 0) │                    0 │       0.601382 │
│ (3//2, 2//3) │                    0 │        1.34188 │
│       (2, 0) │    10728.0+15111.3im │        1.17e-6 │
│    (2, 1//2) │   -10603.0-14053.7im │     1.04044e-6 │
│       (2, 1) │    10112.1+11149.5im │     9.93827e-7 │
└──────────────┴──────────────────────┴────────────────┘
 Channel t
┌──────────────┬──────────────────────┬────────────────┐
│ Field        │ Structure constant   │ Relative error │
├──────────────┼──────────────────────┼────────────────┤
│       (1, 0) │    1.0-5.91106e-14im │    8.71426e-14 │
│       (1, 1) │ -1.12641-0.0491036im │    1.73418e-13 │
│    (3//2, 0) │                    0 │        1.90731 │
│ (3//2, 

### Signature $(0, 0, 0)$, $D^{(s)}_{(1, 0)} = 0$, $D^{(s)}_{(2, 0)} = 0$

In [25]:
fix = [(:s, diag_field, 1), (:s, Field(c, r=1, s=0), 0), (:s, Field(c, r=2, s=0), 0)]
solve(blocks, Sig(0, 0, 0), fix=fix, rmax=2);

 Channel s
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relative error │
├─────────────────┼────────────────────────────┼────────────────┤
│ (P=0.11+0.06im) │                  1.0+0.0im │            0.0 │
│       (1//2, 0) │          13.3099+20.2186im │     1.16449e-6 │
│          (1, 0) │                  0.0+0.0im │            0.0 │
│          (1, 1) │         -8809.16-14776.1im │     1.17125e-6 │
│       (3//2, 0) │     6.82216e+06+460636.0im │     1.69767e-6 │
│    (3//2, 2//3) │ -4.92108e+06-6.33165e+06im │     5.84724e-7 │
│          (2, 0) │                  0.0+0.0im │            0.0 │
│       (2, 1//2) │  6.38691e+06+2.81696e+06im │     8.62956e-7 │
│          (2, 1) │ -1.03563e+08+4.48727e+08im │     1.08336e-6 │
└─────────────────┴────────────────────────────┴────────────────┘
 Channel t
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relat